In [1]:
import cv2
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

In [2]:
mean = np.load('summ.npy')

In [3]:
def generator(data, mean,fp_gender, im_height = 224, im_width = 224, train = True):
    while 1:
        for i in xrange(0, len(data)/5):
            labels = []
            images = np.zeros((5,224,224,3))
            for idx, image_path in enumerate(data[5*i:5*(i+1)]):
                img = cv2.imread('../Data/wiki_crop/' + image_path)
                img = cv2.resize(img, (im_width, im_height))
                images[idx,:,:,:] = img
                if(train):
                    labels.append(int(fp_gender[fp_gender['filepath'] == image_path]['gender']))

            
            images = images - mean
            images = np.swapaxes(images, 1, 3)
            images = np.swapaxes(images, 2, 3)
            if(train):
                labels = np.array(labels)
                yield images, labels
            else:
                yield images

In [4]:
fp_gender = pd.read_csv('../Data/new_gender.csv', sep = '\t').drop('Unnamed: 0', axis = 1)
fp_gender.dropna(inplace = True)
fp = fp_gender['filepath'].as_matrix()
gender = fp_gender['gender'].as_matrix()

In [5]:
fp_train, fp_test, gender_train, gender_test = train_test_split(fp, gender, test_size = 0.2, random_state = 5)
fp_train, fp_cv, gender_train, gender_cv = train_test_split(fp_train, gender_train, test_size = 0.25, random_state = 5)

In [7]:
len(fp_gender)

28453

In [6]:
len(gender_cv)

5691

In [7]:
im_height = 224
im_width = 224
labels = []
images = np.zeros((5691,224,224,3))
for idx, image_path in enumerate(fp_cv):
    img = cv2.imread('../Data/wiki_crop/' + image_path)
    img = cv2.resize(img, (im_width, im_height))
    images[idx,:,:,:] = img
    
    labels.append(int(fp_gender[fp_gender['filepath'] == image_path]['gender']))

labels = np.array(labels)
images = images - mean
images = np.swapaxes(images, 1, 3)
images = np.swapaxes(images, 2, 3)

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, rmsprop
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu',input_shape = (3,224,224), border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add( Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.load_weights('../../Kaggle/Cats and Dogs/Data/vgg16_weights_th_dim_ordering_th_kernels_notop.h5')

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))
model.add(Dense(1, activation='sigmoid'))

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5103)


In [9]:
for layer in model.layers[:18]:
    layer.trainable = False

In [10]:
model.compile(optimizer=rmsprop(lr = 0.0005), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
len(fp_cv)

In [17]:
model.fit_generator(generator(fp_train, mean, fp_gender), samples_per_epoch = 17071,  nb_epoch = 10, validation_data = [images, labels],callbacks= [EarlyStopping(patience=1)])

Epoch 1/10
17070/17071 [============================>.] - ETA: 0s - loss: 0.4465 - acc: 0.8376

c:\Users\AeroDeath\Miniconda2\lib\site-packages\keras-1.2.2-py2.7.egg\keras\engine\training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


17075/17071 [==============================] - 410s - loss: 0.4466 - acc: 0.8376 - val_loss: 0.3698 - val_acc: 0.8696
Epoch 2/10
17075/17071 [==============================] - 412s - loss: 0.3156 - acc: 0.8900 - val_loss: 0.3379 - val_acc: 0.8831
Epoch 3/10
17075/17071 [==============================] - 487s - loss: 0.2867 - acc: 0.8981 - val_loss: 0.2968 - val_acc: 0.9014
Epoch 4/10
17075/17071 [==============================] - 440s - loss: 0.2731 - acc: 0.9043 - val_loss: 0.2992 - val_acc: 0.9027
Epoch 5/10
17075/17071 [==============================] - 438s - loss: 0.2671 - acc: 0.9083 - val_loss: 0.2885 - val_acc: 0.9067
Epoch 6/10
17075/17071 [==============================] - 502s - loss: 0.2606 - acc: 0.9123 - val_loss: 0.2908 - val_acc: 0.9065
Epoch 7/10
17075/17071 [==============================] - 431s - loss: 0.2541 - acc: 0.9162 - val_loss: 0.2836 - val_acc: 0.9100
Epoch 8/10
17075/17071 [==============================] - 493s - loss: 0.2445 - acc: 0.9204 - val_loss: 0.28

In [18]:
model.save('vgg16_gender.h5')

In [10]:
gender_cv.sum()

4134.0